In [2]:
import sys
import os
import cobra
import libsbml
from cobra import Model, Reaction, Metabolite, manipulation, test
import pandas as pd
import csv
import copy
from pprint import pprint
from tabulate import tabulate



#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_manipulation  as mm



/home/rtsantos3/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
ios2164_orig = cobra.io.read_sbml_model('./model/ios2164_orig.xml')

ios2164_orig

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-03


Name,iOS2164
Memory address,0x07f8cfd6383d0
Number of metabolites,1999
Number of reactions,2454
Number of groups,96
Objective expression,1.0*Straw_Biomass - 1.0*Straw_Biomass_reverse_5daed
Compartments,"Cytoplasm, Plastid, Endoplasmic_reticulum, Mitochondrion, Extracellular, Thylakoid, Peroxisome, Vacuole"


In [16]:
ios2164_2cell = cobra.io.read_sbml_model('./model/ios2164_2cell.xml')
ios2164_2cell.reactions.DM_Phloem_BS

#I need to modify the objective function since Starch is being exported from M cell to BS cell. 

Reaction identifier,DM_Phloem_BS
Name,export reaction for phloem sap
Memory address,0x07f8ca5ff88b0
Stoichiometry,0.0183 ala-L_c1 + 0.0958 arg-L_c1 + 0.172 asn-L_c1 + 0.121 asp-L_c1 + 0.189 gln-L_c1 + 0.156 glu-L_c1 + 0.00969 his-L_c1 + 0.0204 ile-L_c1 + 0.0172 leu-L_c1 + 0.0516 lys-L_c1 + 0.0043 phe-L_c1 +... 0.0183 L-Alanine + 0.0958 L-Arginine + 0.172 L-Asparagine + 0.121 L-Aspartate + 0.189 L-Glutamine + 0.156 L-Glutamate + 0.00969 L-Histidine + 0.0204 L-Isoleucine + 0.0172 L-Leucine + 0.0516...
GPR,
Lower bound,0.0
Upper bound,1000000.0


In [6]:
at_core_model = cobra.io.read_sbml_model('./Supplementary_PrevLit/Blaetke & Brautigam/data/2018-23-05-mb-genC3.sbml')
internal_rxns = set(at_core_model.reactions) - set(at_core_model.boundary)

unbalanced_rxns = list()
for rxns in internal_rxns:
    if len(rxns.check_mass_balance()) != 0:
#         print(rxns.id, '\t', rxns.check_mass_balance())
        unbalanced_rxns.append(rxns)
    
    
at_core_model
    

Name,c3_model
Memory address,0x07f8cb0f1c2e0
Number of metabolites,413
Number of reactions,572
Number of groups,0
Objective expression,1.0*Ex_Suc - 1.0*Ex_Suc_reverse_fb96e
Compartments,"Chloroplast, Lumen, Cytosol, Mitochondrion, IntermembraneSpace, Peroxisome"


In [13]:
at_core_model.reactions.cplx4_m.check_mass_balance()

{'charge': -8.0}

In [12]:
at_core_model.boundary

[<Reaction Si_H_c at 0x7f04b360de20>,
 <Reaction Si_H_m at 0x7f04b3794df0>,
 <Reaction Si_H_h at 0x7f04b3602e50>,
 <Reaction Si_H_p at 0x7f04b3794dc0>,
 <Reaction Im_hnu at 0x7f04b37b71c0>,
 <Reaction Im_CO2 at 0x7f04b37b7100>,
 <Reaction Im_H2O at 0x7f04b37b7fd0>,
 <Reaction Im_NH4 at 0x7f04b3807460>,
 <Reaction Im_H2S at 0x7f04b37ab9d0>,
 <Reaction Ex_O2 at 0x7f04b37ab7c0>,
 <Reaction Ex_Ala_c at 0x7f04b37ab310>,
 <Reaction Ex_Ala_h at 0x7f04b3807eb0>,
 <Reaction Ex_Ala_m at 0x7f04b37ab550>,
 <Reaction Ex_Ala_p at 0x7f04b37abf40>,
 <Reaction Ex_Arg_c at 0x7f04b35f3400>,
 <Reaction Ex_Arg_h at 0x7f04b35f3df0>,
 <Reaction Ex_Arg_m at 0x7f04b35f31f0>,
 <Reaction Ex_Arg_p at 0x7f04b35f38e0>,
 <Reaction Ex_Asn_c at 0x7f04b35f3790>,
 <Reaction Ex_Asn_h at 0x7f04b37fce50>,
 <Reaction Ex_Asn_m at 0x7f04b37fc340>,
 <Reaction Ex_Asn_p at 0x7f04b37fcf70>,
 <Reaction Ex_Asp_c at 0x7f04b37fcaf0>,
 <Reaction Ex_Asp_h at 0x7f04b37fc460>,
 <Reaction Ex_Asp_m at 0x7f04b38343a0>,
 <Reaction Ex_Asp_p a

In [24]:
print('internal reactions in B&B core AT model: ', len(internal_rxns))
print('unbalanced rxns in B&B: ', len(unbalanced_rxns))
print('percent unbalanced: ', len(unbalanced_rxns)/len(internal_rxns) * 100)

#IN the case of B&B's model puro charge imbalance yung nagaganap. Yung sa case ko kasi karamihan is mass imbalance talaga.

internal reactions in B&B core AT model:  475
unbalanced rxns in B&B:  88
percent unbalanced:  18.526315789473685


In [25]:
#How about yung core Arabidopsis model nina Shameer et al (2019) -- yung C3 mitochondria
shameer_model = cobra.io.read_sbml_model('../Supplementary_PrevLit/Shameer (2019)/core_model.xml')
internal_rxns = set(shameer_model.reactions) - set(shameer_model.boundary)



Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_FUM_p "FUM">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_FUM_p "FUM">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_FUM_m "FUM">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_FUM_m "FUM">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_VAL_c "VAL">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_VAL_c "VAL">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_2_KETOGLUTARATE_c "2-KETOGLUTARATE">
Use of FORMULA in the notes element is discouraged, 

Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_pVAL_b "pVAL[b]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_pVAL_b "pVAL[b]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_Cytochromes_C_Reduced_mi "Cytochromes_C_Reduced[mi]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_Cytochromes_C_Reduced_mi "Cytochromes_C_Reduced[mi]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_FattyAcid_b "FattyAcid[b]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_FattyAcid_b "FattyAcid[b]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_GLN_c "GLN">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_GLN_c "GLN">
Use of CHARGE in the notes element is discoura

Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_STARCH_p "STARCH[p]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_L_CITRULLINE_p "L-CITRULLINE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_L_CITRULLINE_p "L-CITRULLINE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_CAII_v "CAII[v]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_CAII_v "CAII[v]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_1_KETO_2_METHYLVALERATE_p "1-KETO-2-METHYLVALERATE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_1_KETO_2_METHYLVALERATE_p "1-KETO-2-METHYLVALERATE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_L_CITRULLINE_m "L-CITRULLINE">
Use of FORMULA 

Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_DPG_c "DPG">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_DPG_c "DPG">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_Charged_ILE_tRNAs_c "Charged_ILE_tRNAs[c]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_Charged_ILE_tRNAs_c "Charged_ILE_tRNAs[c]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_GLC_6_P_c "GLC-6-P">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_GLC_6_P_c "GLC-6-P">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_1_L_MYO_INOSITOL_1_P_c "1-L-MYO-INOSITOL-1-P">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_1_L_MYO_INOSITOL_1_P_c "1-L-MYO-INOSITOL-1-P">
Use of CHARGE in the n

Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_NITRITE_c "NITRITE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_NITRITE_c "NITRITE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_NADP_c "NADP">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_NADP_c "NADP">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_ARG_v "ARG">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_ARG_v "ARG">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_2_ACETO_LACTATE_p "2-ACETO-LACTATE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_2_ACETO_LACTATE_p "2-ACETO-LACTATE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_GDP_D_GLUCOSE_

...

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_LYS_pc "LYS_pc">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_CIT_xc "CIT_xc">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_DAHPSYN_RXN_p "DAHPSYN-RXN:2-dehydro-3-deoxyphosphoheptonate aldolase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_CYSPH_RXN_p "CYSPH-RXN:NIL">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_Glycerol_biomass "Glycerol_biomass">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ISOCITDEH_RXN_m "ISOCITDEH-RXN:Isocitrate dehydrogenase (NADP+)">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_2_PERIOD_5_PE

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_NH4_tx "NH4_tx">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ADENYLOSUCCINATE_SYNTHASE_RXN_p "ADENYLOSUCCINATE-SYNTHASE-RXN:adenylosuccinate synthetase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_RXN_3523_p "RXN-3523_p">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_Mg_biomass "Mg_biomass">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FERRIC_CHELATE_REDUCTASE_RXN_c "FERRIC-CHELATE-REDUCTASE-RXN:Ferric-chelate reductase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_OAA_xc "OAA_xc">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ARGININE_TRNA_LIGASE_RXN_c "ARGININE--TRNA-LIGASE-RXN:Arginine--tRNA ligase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_CYSTAGLY_RXN_p "CYSTAGLY-RXN:Cystathionine &amp;gamma;-lyase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_GLY3KIN_RXN_p "GLY3KIN-RXN:Glycerate kinase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FORMYLTHFDEFORMYL_RXN_p "FORMYLTHFDEFORMYL-RXN:Formyltetrahydrofolate deformylase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PHOSGLYPHOS_RXN_p "PHOSGLYPHOS-RXN:Phosphoglycerate kinase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_2_PERIOD_7_PERIOD_7_PERIOD

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_GLUC1PURIDYLTRANS_RXN_c "GLUC1PURIDYLTRANS-RXN:UTP--glucose-1-phosphate uridylyltransferase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_MYO_INOSITOL_OXYGENASE_RXN_c "MYO-INOSITOL-OXYGENASE-RXN:Myo-inositol oxygenase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_CYSTATHIONINE_BETA_LYASE_RXN_p "CYSTATHIONINE-BETA-LYASE-RXN:Cystathionine &amp;beta;-lyase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_6PFRUCTPHOS_RXN_p "6PFRUCTPHOS-RXN:NIL">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PRIBFAICARPISOM_RXN_p "PRIBFAICARPISOM-RXN:N-(5&#039;-phospho-D-ribosylformimino)-5-amino-1-(5&#039;&#039;-phosphoribosyl)-4- imidazole carboxamide isomer

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PYRROLINECARBREDUCT_RXN_NADP_c "PYRROLINECARBREDUCT-RXN">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PYRROLINECARBREDUCT_RXN_NAD_c "PYRROLINECARBREDUCT-RXN">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_unlProtHYPO_c "unlProtHYPO_c">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_HS_pc "plastidic HS diffusion">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ATP_CITRATE_PRO_S_LYASE_RXN_c "ATP-CITRATE-PRO-S--LYASE-RXN:ATP-citrate lyase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_TRP_pc "TRP_pc">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBoun

In [27]:
shameer_model

Name,TomCoreOxford
Memory address,0x07f7489d3f460
Number of metabolites,568
Number of reactions,653
Number of groups,92
Objective expression,0.0
Compartments,"Plastid, Mitochondrion, Cytoplasm, Vacuole, Biomass, Peroxisome, Pool, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


In [26]:

unbalanced_rxns = list()
for rxns in internal_rxns:
    if len(rxns.check_mass_balance()) != 0:
#         print(rxns.id, '\t', rxns.check_mass_balance())
        unbalanced_rxns.append(rxns)
    
print('internal reactions in Shameer core AT model: ', len(internal_rxns))
print('unbalanced rxns in Shameer AT model: ', len(unbalanced_rxns))
print('percent unbalanced: ', len(unbalanced_rxns)/len(internal_rxns) * 100)

#IN the case of B&B's model puro charge imbalance yung nagaganap. Yung sa case ko kasi karamihan is mass imbalance talaga.

internal reactions in Shameer core AT model:  637
unbalanced rxns in Shameer AT model:  313
percent unbalanced:  49.13657770800628


In [32]:
shameer_model.reactions.RXN0_5224_c

Reaction identifier,RXN0_5224_c
Name,RXN0-5224:NIL
Memory address,0x07f74899020d0
Stoichiometry,CARBON_DIOXIDE_c + WATER_c <=> HCO3_c + PROTON_c CARBON-DIOXIDE + WATER <=> HCO3 + PROTON
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Notes:
Weirdly enough the Shameer model is even much more imbalanced compared dun sa current model ko, which I believe is more of an encoding issue as opposed with actual mass-charge violations, considering that there weren't much changes done by the author in their supplementary scripts. 
Meanwhile, the core Arabidopsis model used by B&B doesn't have much mass-charge violations owing to the small size of the model.
So far after implementing constraints on cofactor recycling (based on the Boolean logic algorithm formulated by Lakshmanan in the original iOS2164) there seems to be limited evidence of any type II or type III internal loops with pFBA. Furthermore, there is concordance with the scale of flux values when comparing with the experimentation by Shameer et al in both the purely linear diel model and the hybrid kinetic setup (2018 and 2019), indicating that the model produces relatively realistic ATP rates and other things. 